In [7]:

from langchain_community.vectorstores import Weaviate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOllama

import weaviate
from langchain.globals import set_llm_cache
from langchain.cache import RedisCache
import redis

REDIS_URL = "redis://localhost:6379/0"

redis_client = redis.Redis.from_url(REDIS_URL)
set_llm_cache(RedisCache(redis_client))


client = weaviate.Client(
url="http://localhost:8080",
)

vectorstore = Weaviate(client, 
                    "GRP", 
                    "content")

retriever = vectorstore.as_retriever()



# RAG prompt
template = """You're an Friendly AI assistant, your name is Claro, you can make normal conversations in a friendly manner, and also provide Answer the question based on the following context make sure it sounds like human and official assistant:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)




# RAG
model = ChatOllama(model="openhermes:7b-mistral-v2-q8_0")
# model = ChatOllama(model="falcon:40b-instruct-q4_1")
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    # | StrOutputParser()
)



# Add typing for input
class Question(BaseModel):
    __root__: str


chain = chain.with_types(input_type=Question)



/home/amaithi/anaconda3/envs/lang/lib/python3.10/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.1. The latest version is 4.5.4.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [8]:
result = chain.invoke('what is mediwave')

In [9]:
result

AIMessage(content="Mediwave Digital is a healthcare technology and clinical research company that focuses on combining user-centered design with agile development. They specialize in creating digital products and services that deliver delightful customer experiences, while also addressing the world's needs and making a meaningful impact. Mediwave works closely with clients to understand their needs and provide tailored solutions, ensuring satisfaction throughout the customer journey.")

In [ ]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [ ]:
from grp_food_crew_ai.grp_food_crewai import food_crew


result = food_crew(input='how to make vanilla sponge cake give me the receipe only')


Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "ParameterLocation.HEADER" for parameter Accept. Valid values are ['query', 'path'] Ig

 [DEBUG]: Working Agent: Crew Manager
 [DEBUG]: [Crew Manager] Task output: None
{'final_output': None, 'tasks_outputs': [None]}


> Entering new CrewAgentExecutor chain...


 I should delegate the task of finding and providing the recipe for a vanilla sponge cake to the Ingredient Expert since they have extensive knowledge about ingredients and their usage in various recipes.

Action: Delegate work to co-worker
Action Input: {"coworker": "Ingredient Expert", "task": "Find and provide the recipe for a vanilla sponge cake", "context": "This is a request from a user who wants to make a vanilla sponge cake. They only want the receipe."} 


Error executing tool. Co-worker mentioned not found, it must to be one of the following options:
- Master Chef
- Nutrition Expert
- Ingridient Expert
- Image analyzing Expert
- grocery Expert
- wine analyzing Expert
- food joke and trivia Expert 
- Head of the crew


 Thought: I apologize for the error in my previous action. I should have delegated the task to the Master Chef instead, as they are responsible for preparing and cooking various dishes, including vanilla sponge cake.

Action: Delegate work to co-worker
Action In

/home/amaithi/projects/weaviate/langgraph-final/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


 

ERROR making request

 Thought: It seems I encountered an error while trying to use Spoonacular API's searchRecipesByIngredients tool. Let me try using a different tool or provide a recipe from my knowledge base.

Action: None
Action Input: None 

Action 'None' don't exist, these are the only available Actions: spoonacular_API.searchRecipes: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: Search through hundreds of thousands of recipes using advanced filtering and ranking. NOTE: This method combines searching by query, by ingredients, and by nutrients into one endpoint.
spoonacular_API.searchRecipesByIngredients: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: 
Ever wondered what recipes you can cook with the ingredients you have in your fridge or pantry? This endpoint lets you find recipes that either maximize the usage of ingredients you have at hand (pre shopping) or minimize

/home/amaithi/projects/weaviate/langgraph-final/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/amaithi/projects/weaviate/langgraph-final/.venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


 

Please provide an API_RESPONSE for me to process and create a human-understandable synthesis.

 Thought: I need to wait for the API response before I can create a human-understandable synthesis of the search results.

Action: None
Action Input: None 

Action 'None' don't exist, these are the only available Actions: spoonacular_API.searchRecipes: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: Search through hundreds of thousands of recipes using advanced filtering and ranking. NOTE: This method combines searching by query, by ingredients, and by nutrients into one endpoint.
spoonacular_API.searchRecipesByIngredients: I'm an AI from spoonacular API. Instruct what you want, and I'll assist via an API with description: 
Ever wondered what recipes you can cook with the ingredients you have in your fridge or pantry? This endpoint lets you find recipes that either maximize the usage of ingredients you have at hand (pre shopping) or mini

In [2]:
print(result)

{'final_output': None, 'tasks_outputs': [None]}


In [4]:
from grp_food_crew_ai.grp_food_crewai import food_crew


result = food_crew(input='how to make vanilla sponge cake, give me the receipe')

print(food_crew.usage_metrics())

Overriding of current TracerProvider is not allowed


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 				**Task**: find and provide the appropriate soultion to the user needs.
           **Description**: you are an helpfull assuistane.    
            **Parameters**: 
            - user need: how to make vanilla sponge cake, give me the receipe


> Entering new CrewAgentExecutor chain...
First, I will delegate the task of creating the recipe for vanilla sponge cake to the Master Chef. Then, I'll ask the Nutrition Expert about the ingredients used in the recipe and their nutritional value. Finally, I'll gather information on how to make the sponge cake from the Master Chef and present it as the final answer.

Action: Delegate work to co-worker
Action Input: {"coworker": "Master Chef", "task": "create vanilla sponge cake recipe"} 

I encountered an error while trying to use the tool. This was the error: AgentTools.delegate_work() missing 1 required positional argument: 'context'.
 Tool Delegate work to co-worker accepts these

KeyboardInterrupt: 

In [2]:
print(result)

To make this chocolate cake, first preheat the oven to 350°F (180°C) and grease an 8-inch round cake pan with cooking spray or butter. In a medium bowl, whisk together flour, cocoa powder, baking powder, baking soda, and salt. In a large bowl, beat sugar and oil until combined, then add the egg and mix well. Stir in milk and vanilla extract. Gradually add the dry ingredients to the wet mixture, stirring until just combined. Slowly pour in boiling water and stir until the batter is smooth. Pour the batter into the prepared cake pan and bake for 30-35 minutes or until a toothpick inserted into the center comes out clean. Let the cake cool before serving. To ensure the best results, use fresh ingredients and follow the recipe's instructions carefully.


Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values 

RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
a = 5 
a

5

# TRavel

In [1]:
from langchain.globals import set_debug, set_verbose

# set_verbose(True)
# set_debug(True)

import os

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_experimental.llms.ollama_functions import OllamaFunctions

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning) 

import warnings
warnings.filterwarnings("ignore")

# members = ["Food_crew", "General_conversation", "General_other", "Mediwave_rag", "Travel_crew"]


members = ["Food_crew", "General_conv", "General_other", "Mediwave_rag", "Travel_crew"]

system_prompt = (
    """You are a supervisor tasked with managing a conversation between the
     following workers:  {members}. Given the following user request,"
     respond with the worker to act next. 
     
     if the user asks anything related to food, receipies, and it's related stuffs use 'Food_crew',
    if the user asks anything related to mediwave and it's related stuffs use 'Mediwave_rag',    
    if the user makes conversation, jokes and funny conversations then use 'General_conv',
    if the user asks anything related to weather, time, wikipedia and it's related stuffs use 'General_other',
    if the user asks anything related to travel, exploration, city tour and it's related stuffs use 'Travel_crew'
        
    Each worker will perform a
     task and respond with their results and status. When finished,
    respond with FINISH."""
)

# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role to act",
    
    "parameters": {
        "type": "object",        
        "properties": {
            "next": {
                "type": "string",
                "enum": f"{options}",
            }
        },
        "required": ["next"],
    },
}


DEFAULT_SYSTEM_TEMPLATE = """You have access to the following tools:

{tools}

You must always select one of the above tools and respond with only a JSON object matching the following schema:

{{
  "tool": "route",
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}
""" 


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

llm = OllamaFunctions(
    model=os.environ['LLM'],
    tool_system_prompt_template=DEFAULT_SYSTEM_TEMPLATE
    )


def supervisor_node(state):

    print(state)
    
    supervisor_chain = (
        prompt
        | llm.bind(functions=[function_def], function_call={"name": "route"})
        | JsonOutputFunctionsParser()
    )
        
    result = supervisor_chain.invoke(state)
    
    return result





import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str



from grp_travel_crew_ai.grp_travel_crewai import travel_crew

from grp_RAG1.grp_rag1_rag import mediwave_rag

from grp_others.grp_others_graph import grp_other_def as gen_others

from grp_food_crew_ai.grp_food_crewai import food_crew

from grp_Gen_Conv.grp_gen_conv_chain import general_conversation



workflow = StateGraph(AgentState)

workflow.add_node("Food_crew", food_crew)
workflow.add_node("General_conv", general_conversation)
workflow.add_node("General_other", gen_others)
workflow.add_node("Mediwave_rag", mediwave_rag)
workflow.add_node("Travel_crew", travel_crew)

workflow.add_node("supervisor", supervisor_node)




for member in members:
    
    if member == 'Mediwave_rag':
        continue
    
    
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
    

# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes


conditional_map = {k: k for k in members}



conditional_map["FINISH"] = END
# conditional_map['supervisor'] ='supervisor'

workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

# Finally, add entrypoint
workflow.set_entry_point("supervisor")
workflow.set_finish_point('Mediwave_rag')
workflow.set_finish_point('General_conv')


graph = workflow.compile()


Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['query', 'path'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values 

In [2]:
result = graph.invoke({
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in kodaikanal and give me a 3 day itenary, start from pondicherry and return pondicherry after third day and do suggest me a route for my travel"
            )
        ],
    })

{'messages': [HumanMessage(content='suggest some good spots to visit during summer in kodaikanal and give me a 3 day itenary, start from pondicherry and return pondicherry after third day and do suggest me a route for my travel')], 'next': None}


> Entering new CrewAgentExecutor chain...
 I need to find appropriate travel solutions for a user who wants to visit some good spots in Kodaikanal during summer and create a 3-day itinerary starting from Pondicherry and returning there after the third day. I should also suggest a route for their travel.

Action: Delegate work to Local Tour Guide
Action Input: {"task": "Help me identify popular tourist spots in Kodaikanal during summer", "context": "Kodaikanal, summer, tourist spots"} 

Action 'Delegate work to Local Tour Guide' don't exist, these are the only available Actions: Delegate work to co-worker: Delegate work to co-worker(coworker: str, task: str, context: str) - Delegate a specific task to one of the following co-workers: ['Expert 

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {
  "tool": "Travel_crew",
  "tool_input": {
    "next": "General_other"
  }
}

  

  

  

  

  

  

  

  

  

  


In [2]:
result = graph.invoke({
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in ooty and give me a 3 day itenary, start from pondicherry and return pondicherry after third day"
            )
        ],
    })

{'messages': [HumanMessage(content='suggest some good spots to visit during summer in ooty and give me a 3 day itenary, start from pondicherry and return pondicherry after third day')], 'next': None}


> Entering new CrewAgentExecutor chain...
 I need more information to provide an accurate solution for the user's travel needs in Ooty during summer. I will delegate this task to the City Selection Expert.

Action: Delegate work to co-worker
Action Input: {"coworker": "City Selection Expert", "task": "Research and identify top attractions and a 3-day itinerary for visiting Ooty during summer, starting from Pondicherry and returning there after the third day"} 

I encountered an error while trying to use the tool. This was the error: AgentTools.delegate_work() missing 1 required positional argument: 'context'.
 Tool Delegate work to co-worker accepts these inputs: Delegate work to co-worker(coworker: str, task: str, context: str) - Delegate a specific task to one of the following co-worke

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {"tool": "Travel_crew",
"tool_input": {
"destination": "Ooty",
"duration": "3 days",
"starting_point": "Pondicherry",
"itinerary": ["Dodabetta Peak", "Mudumalai Wildlife Sanctuary", "Botanical Gardens", "Emerald Lake", "Pykara Falls", "Coonoor"]
}}

In [ ]:
"The user's travel needs in Ooty during summer can be met by visiting several top attractions over a 3-day itinerary, starting and ending in Pondicherry. Here is the detailed plan:\n\nDay 1:\n- Visit Dodabetta Peak, the highest point in Ooty, offering breathtaking views of the surrounding mountains and valleys.\n- Explore Mudumalai Wildlife Sanctuary, a large protected area known for its diverse wildlife population, including elephants, tigers, and various bird species.\n\nDay 2:\n- Spend the morning at the Botanical Gardens, home to a vast collection of exotic plants, flowers, and trees.\n- In the afternoon, visit Emerald Lake, a serene and picturesque lake nestled in the heart of Ooty.\n- End the day by relaxing at Ooty Lake, enjoying a peaceful boat ride or taking a leisurely walk around the lake.\n\nDay 3:\n- Begin the day with a visit to Pykara Falls, located approximately 20 km from Ooty, known for its stunning beauty and the surrounding lush greenery.\n- In the afternoon, head towards Coonoor, a nearby hill station famous for its tea gardens. Take a tour of the gardens and learn about the tea-making process.\n\nBy following this itinerary, the user will get to experience the best of Ooty's natural beauty, cultural attractions, and local experiences during their summer vacation."

In [2]:
result = graph.invoke({
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in mysore and give me a 3 day itenary, start from pondicherry and return pondicherry after third day"
            )
        ],
    })

{'messages': [HumanMessage(content='suggest some good spots to visit during summer in mysore and give me a 3 day itenary, start from pondicherry and return pondicherry after third day')], 'next': None}


> Entering new CrewAgentExecutor chain...
 I need to find suitable travel spots in Mysore during summer and create a three-day itinerary starting from Pondicherry. I should delegate this task to a co-worker who has expertise in travel planning and city selection.

Action: Delegate work to co-worker
Action Input: {'coworker': 'Travel agency manager', 'task': 'Find suitable travel spots in Mysore during summer and create a three-day itinerary starting from Pondicherry', 'context': 'The user is looking for suggestions on places to visit during summer in Mysore, and they want a 3-day itinerary that starts from Pondicherry and returns there after the third day.'}

> Entering new CrewAgentExecutor chain...
 Thought: I now have all the necessary information to provide a great answer

Final An

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {"tool": "Travel_crew",
"tool_input": {
"destination": "Mysore, India",
"activity": ["visiting Chamundeshwari Temple", "Brindavan Gardens", "Mysore Palace", "Sri Chamarajendra Zoo and Museum", "St. Philomena's Church", "Ranganathittu Bird Sanctuary"],
"duration": "3 days",
"start_location": "Pondicherry, India"
}}

In [ ]:
"Here's a detailed response summarizing key findings about the given context and information that could be relevant to it: The user is looking for travel suggestions in Mysore during summer and requires a three-day itinerary starting from Pondicherry. Based on your request, I have delegated this task to our Travel agency manager. Here's the suggested itinerary:\n\nDay 1:\n- Depart from Pondicherry early morning by road, which is approximately a 4-hour drive.\n- Visit the Chamundeshwari Temple located on Chamundi Hill, known for its religious significance and beautiful views of Mysore city.\n- Head to Brindavan Gardens, located in the Krishnarajasagar Dam area, famous for its terrace gardens, fountains, and musical fountain show.\n- Overnight stay at a hotel in Mysore.\n\nDay 2:\n- Visit the Mysore Palace, a grand architectural marvel open to public viewing during summer months. Don't forget to check out the famous Dussehra durbar hall.\n- Explore the Sri Chamarajendra Zoo and Museum located in the heart of Mysore city. The zoo is home to a wide variety of animals, and the museum exhibits artifacts related to the history and culture of Mysore.\n- Visit the St. Philomena's Church, an impressive Roman Catholic basilica known for its neo-Gothic architecture.\n- Overnight stay at a hotel in Mysore.\n\nDay 3:\n- Depart from Mysore early morning and head back to Pondicherry by road.\n- En route, stop at the Ranganathittu Bird Sanctuary located near Srirangapatna, famous for its diverse bird population and scenic beauty.\n- Arrive in Pondicherry late afternoon/evening and complete your journey.\n\nPlease note that travel times may vary depending on traffic conditions, so it's always a good idea to leave early. Additionally, make sure to check the opening hours of each attraction before planning your visit. Let me know if you need any further assistance with this itinerary or if there are any modifications you would like me to make."

In [3]:
for s in graph.stream(input=
    {
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in mysore and give me a 3 day itenary, start from pondicherry and return pondicherry after third day"
            )
        ],
    },
    config= {"recursion_limit": 100},
    
):
    if "__end__" not in s:
        print(s)
        print("---")


{'supervisor': {'next': 'Travel_crew'}}
---


> Entering new CrewAgentExecutor chain...
 To find and provide an appropriate travel solution for the user's summer exploration in Mysore, starting from Pondicherry and returning after three days, I should first gather more specific information about the points of interest and travel routes.

Action: Ask question to City Selection Expert
Action Input: {'question': 'Can you recommend some popular tourist spots in Mysore that would be suitable for summer exploration?', 'context': 'Mysore, Summer'} 

Action 'Ask question to City Selection Expert' don't exist, these are the only available Actions: Delegate work to co-worker: Delegate work to co-worker(coworker: str, task: str, context: str) - Delegate a specific task to one of the following co-workers: ['Expert Travel Agent', 'City Selection Expert', 'Local Tour Guide', 'Travel agency manager']
The input to this tool should be the coworker, the task you want them to do, and ALL necessary contex

2024-03-28 15:54:24,884 - 139627801343552 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


{'supervisor': {'next': 'Travel_crew'}}
---


> Entering new CrewAgentExecutor chain...
 To find the appropriate travel solution for the user needs, I need to gather more information about the recommended places to visit during summer in Mysore, along with their proximity to Pondicherry and a suggested itinerary.

Action: Ask question to City Selection Expert
Action Input: {'question': 'Can you suggest some popular tourist spots to visit in Mysore during summer?', 'context': 'The user is looking for recommendations on places to visit during summer in Mysore, as part of a three-day itinerary starting from Pondicherry and returning there after the third day.'} 

Action 'Ask question to City Selection Expert' don't exist, these are the only available Actions: Delegate work to co-worker: Delegate work to co-worker(coworker: str, task: str, context: str) - Delegate a specific task to one of the following co-workers: ['Expert Travel Agent', 'City Selection Expert', 'Local Tour Guide', 'Trave

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {
  "tool": "Travel_crew",
  "tool_input": {
    "next": "General_other"
  }
}

 

 

 

 

 

 

 

 

 

 


In [2]:
for s in graph.stream(input=
    {
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in pondicherry and give me a 2 day itenary"
            )
        ],
    },
    config= {"recursion_limit": 100},
    
):
    if "__end__" not in s:
        print(s)
        print("---")


{'supervisor': {'next': 'Travel_crew'}}
---


> Entering new CrewAgentExecutor chain...
 I should delegate the task of finding specific spots to visit during summer in Pondicherry and creating a 2-day itinerary to my City Selection Expert, as they have extensive knowledge about various travel destinations and can create effective itineraries.

Action: Delegate work to co-worker
Action Input: {'coworker': 'City Selection Expert', 'task': 'Find specific spots to visit during summer in Pondicherry and create a 2-day itinerary'} 

I encountered an error while trying to use the tool. This was the error: AgentTools.delegate_work() missing 1 required positional argument: 'context'.
 Tool Delegate work to co-worker accepts these inputs: Delegate work to co-worker(coworker: str, task: str, context: str) - Delegate a specific task to one of the following co-workers: ['Expert Travel Agent', 'City Selection Expert', 'Local Tour Guide', 'Travel agency manager']
The input to this tool should be the 

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {"tool": "Travel_crew",
"tool_input": {}}
































In [2]:
for s in graph.stream(input=
    {
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in pondicherry and give me a 2 day itenary"
            )
        ],
    },
    config= {"recursion_limit": 100},
    
):
    if "__end__" not in s:
        print(s)
        print("---")


{'supervisor': {'next': 'Travel_crew'}}
---


> Entering new CrewAgentExecutor chain...
 To find and provide an appropriate travel solution for the user's needs in Pondicherry during summer, I should first gather more information about the city and its popular tourist spots. I will delegate this task to the City Selection Expert.

Action: Delegate work to co-worker
Action Input: {'coworker': 'City Selection Expert', 'task': 'Research popular tourist spots in Pondicherry during summer', 'context': ''}

Thought: Once I have the list of popular tourist spots, I will need to create a 2-day itinerary for the user. I will delegate this task to the Local Tour Guide.

Action: Delegate work to co-worker
Action Input: {'coworker': 'Local Tour Guide', 'task': 'Create a 2-day itinerary for Pondicherry with the given tourist spots', 'context': ''} Thought: I apologize for the mistake in my previous action input. I should provide a valid dictionary format as follows:

Action: Delegate work to co-wor

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {"tool": "Travel_crew", "tool_input": {}}

 

 

 

 

 

 

 

 

 

 


In [3]:
for s in graph.stream(input=
    {
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in pondicherry and give me a 2 day itenary"
            )
        ],
    },
    config= {"recursion_limit": 100},
    
):
    if "__end__" not in s:
        print(s)
        print("---")


{'supervisor': {'next': 'Travel_crew'}}
---
 [DEBUG]: Working Agent: Crew Manager


> Entering new CrewAgentExecutor chain...
 I need more information to provide an accurate travel solution for the user. I should delegate this task to my City Selection Expert as they have extensive knowledge about various travel destinations and their attractions.

Action: Delegate work to co-worker
Action Input: {'coworker': 'City Selection Expert', 'task': 'Research popular summer spots in Pondicherry and create a 2-day itinerary.', 'context': "The user is looking for suggestions on good places to visit during summer in Pondicherry, India, and they would also like a 2-day itinerary."}

> Entering new CrewAgentExecutor chain...
 I should research popular summer spots in Pondicherry and create a 2-day itinerary for the user.

Action: Search the internet
Action Input: {"query": "Popular summer spots in Pondicherry, India 2-day itinerary"} 

Title: 2 Days Pondicherry (Top Places to visit in ...
Link: htt

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {
  "tool": "Travel_crew",
  "tool_input": {
    "next": "Provide suggestions for a 2-day itinerary in Pondicherry and summer spots to visit"
  }
}

In [3]:
result = graph.invoke({
        "messages": [
            HumanMessage(
                content="suggest some good spots to visit during summer in pondicherry and give me a 2 day itenary"
            )
        ],
    })

2024-03-28 14:30:54,814 - 139923841603136 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
 I should delegate the task of finding specific spots to visit during summer in Pondicherry and creating a 2-day itinerary to my City Selection Expert. I can provide them with all necessary context to execute the task effectively.

Action: Delegate work to co-worker
Action Input: {'coworker': 'City Selection Expert', 'task': 'Find specific spots to visit during summer in Pondicherry and create a 2-day itinerary', 'context': "The user is looking for recommendations on places to visit during their summer trip to Pondicherry. They have specifically requested a 2-day itinerary. Pondicherry is located in the union territory of Puducherry, India, and is known for its French colonial influences. Some popular tourist attractions include the Aurobindo Ashram, Sri Aurobindo Mandir, and the Promenade Beach. The city also has a rich history and cultural heritage with various temples and churches."}

Thought: Once I have received the information from my C

2024-03-28 14:37:01,364 - 139923841603136 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
 I need more information to provide an accurate travel solution for the user's needs in Pondicherry during summer. I should delegate this task to a co-worker who is an expert in city selection.

Action: Delegate work to co-worker
Action Input: {'coworker': 'City Selection Expert', 'task': 'Research and provide information about popular spots to visit in Pondicherry during summer and create a 2 day itinerary.', 'context': 'The user has requested suggestions for good spots to visit during summer in Pondicherry and a 2 day itinerary. The city selection expert is the best person to gather this information.'}

> Entering new CrewAgentExecutor chain...
 I need to find popular spots to visit in Pondicherry during summer and create a 2-day itinerary.

Action: Search the internet - "popular places to visit in Pondicherry during summer"

Action Input: {} 

Action 'Search the internet - "popular places to visit in Pondicherry during summer"' don't exist

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {"tool": "Travel_crew", "tool_input": {}}
































In [4]:
result

NameError: name 'result' is not defined

In [5]:
graph.invoke({
        "messages": [
            HumanMessage(
                content="what is the time now"
            )
        ],
    })

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))